# K-means Clustering

The objective of this notebook is to develop a generic code to run K-Means clustering on numerical csv datasets. The final goal is to find clusters and report the average value for each feature by cluster in order to analyze the differences between them.

First, we will read the data:

In [3]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/PDA_Data.csv")

And will take a look:

In [5]:
df.show()

We will get rid of the ID column since it is just an index:

In [7]:
df = df.select("Innovator","Use Message","Use Cell","Use PIM","Inf Passive","Inf Active","Remote Access","Share Inf","Monitor","Email","Web","Mmedia","Ergonomic","Monthly","Price","Age","Education","Income","Construction","Emergency","Sales")

df.show()

In [8]:
df.printSchema

Now, we need to prepare our data to train the K-Means model. For this we need to transform each row into a DenseVector and then we have to standardize the values. To accomplish that we will define the function 'prepare_data' that takes as input a dataframe and will return a standardized DenseVector:

In [10]:
def prepare_data(df):
  
  from pyspark.mllib.feature import StandardScaler, StandardScalerModel
  from pyspark.mllib.util import MLUtils
  from pyspark.mllib.regression import LabeledPoint
  
  #We use the LabeledPoint object to transform the dataframe to a DenseVector
  data = df.rdd.map(lambda line: LabeledPoint(0,[line[0:]]))
  #We take just the features
  features = data.map(lambda x: x.features)
  #We fit and transform using the StandardScaler function
  scaler = StandardScaler(withMean=True, withStd=True).fit(features)
  features_scale = scaler.transform(features)
  return features_scale

Using the function to create get the Standardized DenseVector:

In [12]:
features_scaled = prepare_data(df)
features_scaled.collect()

The next step is to train the K-Means model to get the clusters. For this we will create a function that takes as input the features in DenseVector format, the number of clusters that you want to find and a seed in order to be able to replicate the results. The output is the trained K-means model and the cluster of each observation found by the model:

In [14]:
def kmeans_model(features, n_clusters, seed):
  
  from pyspark.mllib.clustering import KMeans, KMeansModel
  
  km_model = KMeans.train(features, n_clusters, maxIterations=10, initializationMode="random",seed=seed)
  clusters = km_model.predict(features)
  
  #from math import sqrt

  #def error(point):
  #    center = km_model.centers[km_model.predict(point)]
  #    return sqrt(sum([x**2 for x in (point - center)]))

  #WSSSE = features.map(lambda point: error(point)).reduce(lambda x, y: x + y)
  #print("Within Set Sum of Squared Error = " + str(WSSSE))
  
  return km_model, clusters

We call the function to train the K-means model using the Standardized DenseVector that we created before, setting 5 clusters to be found, and 1234 as random seed. Then we display the clusters found by the model:

In [16]:
km_model, clusters = kmeans_model(features_scaled, 5, 1234)
clusters.collect()

We create this function to evaluate the Within Set Sum of Squared Error (WSSSE). The lower WSSSE, the better. This is a useful measure if you want to try different number of clusters and see which number fits the data better:

In [18]:
def WE(kmeans_model,features):
  
  from math import sqrt

  def error(point):
      center = kmeans_model.centers[kmeans_model.predict(point)]
      return sqrt(sum([x**2 for x in (point - center)]))

  WSSSE = features.map(lambda point: error(point)).reduce(lambda x, y: x + y)
  #print("Within Set Sum of Squared Error = " + str(WSSSE))
  return WSSSE

In [19]:
WE(km_model,features_scaled)

Now, we will put the cluster and the original dataframe together:

In [21]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
def df_with_clusters(df,k_means_clusters):
  
  from pyspark.sql import Row

  #We crete a dataframe columns with the clusters
  row = Row('Cluster')
  cluster = k_means_clusters.map(row).toDF()
  
  from pyspark.sql.types import StructType
  from pyspark.sql.types import DataType
  
  #We create the structure of a new dataframe that will include an index value to join with the clusters column
  Structure = df.schema[0:len(df.columns)]
  Structure = Structure.add("index", IntegerType(),True)
  
  #Create the dataframe with index
  df_index = df.rdd.zipWithIndex().map(lambda (row, columnindex): row + (columnindex,)).toDF(StructType(Structure))
  
  #We create the structure type of a new dataframe for the clusters and an index to join with the features dataframe
  Structure_type = StructType([cluster.schema[0],StructField("index", IntegerType(),True)])
  
  #We create the dataframe with clusters and index
  cluster_index = cluster.rdd.zipWithIndex().map(lambda (row, columnindex): row + (columnindex,)).toDF(Structure_type)

  #We join the original feautres with the clusters and drop the index
  df_final = df_index.join(cluster_index, df_index.index==cluster_index.index)
  df_final = df_final.drop('index')
  
  return df_final

Now that we have our function to join dataframes, we join the original dataframe with the clusters obtained using the K-Means model:

In [23]:
df_final = df_with_clusters(df,clusters)
df_final.show()

Finally, we calculate the average value of each feature for each cluster:

In [25]:
def avg_by_cluster(df):
  
  exprs = {x: "mean" for x in df.columns}
  df_by_cluster = df_final.groupBy("Cluster").agg(exprs)
  
  return df_by_cluster

In [26]:
df_avg_by_cluster = avg_by_cluster(df_final)
df_avg_by_cluster.show(3)

Saving the results:

In [28]:
df_avg_by_cluster.write.option("header", True).csv("/FileStore/tables/clustering_results.csv")

Now you can use the results to check differences among clusters. For example:

In [30]:
df_avg_by_cluster.select('Cluster','avg(Age)','avg(Price)','avg(Innovator)').show()

############################################################################################################################################################

Working Notebook: This part contains some of the code used in the functions displayed above.

In [33]:
from pyspark.sql import Row

row = Row('Cluster')
cluster = clusters.map(row).toDF()
cluster.show()

In [34]:
df.rdd.collect()
cluster.rdd.collect()

df.rdd.zip(cluster.rdd).take(5)

In [35]:
from pyspark.sql.types import StructType

def zip_df(l, r):
    return l.rdd.zip(r.rdd).map(lambda x: (x[0][0],x[0][1],x[1][0])).toDF(StructType([l.schema[0],l.schema[1],r.schema[0]]))

df_final = zip_df(df, cluster.select('Cluster'))
df_final.show()

In [36]:
from pyspark.sql.types import StructType
from pyspark.sql.types import DataType

In [37]:
from pyspark.sql.types import *

Structure = df.schema[0:21]
Structure = Structure.add("index", IntegerType(),True)
Structure                       

In [38]:
df.rdd.zipWithIndex().map(lambda (row, columnindex): row + (columnindex,)).toDF(StructType(Structure)).show(5)

In [39]:
df_index = df.rdd.zipWithIndex().map(lambda (row, columnindex): row + (columnindex,)).toDF(StructType(Structure))

In [40]:
Structure_type = StructType([cluster.schema[0],StructField("index", IntegerType(),True)])
Structure_type

In [41]:
cluster.rdd.zipWithIndex().map(lambda (row, columnindex): row + (columnindex,)).toDF(Structure_type).show(5)

In [42]:
cluster_index = cluster.rdd.zipWithIndex().map(lambda (row, columnindex): row + (columnindex,)).toDF(Structure_type)

In [43]:
from pyspark.sql.functions import *

df_final = df_index.join(cluster_index, df_index.index==cluster_index.index)
df_final.show(3)

In [44]:
df_final = df_final.select("Innovator","Use Message","Use Cell","Use PIM","Inf Passive","Inf Active","Remote Access","Share Inf","Monitor","Email","Web","Mmedia","Ergonomic","Monthly","Price","Age","Education","Income","Construction","Emergency","Sales","Cluster")

df_final.show()

In [45]:
df.columns

In [46]:
exprs = {x: "mean" for x in df.columns}

df_by_cluster = df_final.groupBy("Cluster").agg(exprs)

df_by_cluster.show(3)

In [47]:
df_by_cluster.toPandas().to_csv('clustering_results.csv')